<a href="https://colab.research.google.com/github/flaviopradoaquino/TCC_MBA_USP_ESALQ/blob/main/TCC_USP_Teste_Estacionaridade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação e Importação das Bibliotecas**

In [1]:
!pip install -q yfinance

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import statsmodels.tsa.stattools
import plotly.express as px
from datetime import date, timedelta

# **Construção do Dataset**

In [3]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')

In [8]:
from datetime import datetime 
dt_atual = date.today()
td = timedelta(1095)
dt_final = datetime(2023, 3, 3)

In [9]:
### Data de referência menos 1095 dias
df_teste_est = yf.download("BOVA11.SA", start=(dt_final-td))['Close']

### Data atual menos 1095 dias
# df = yf.download("BOVA11.SA", start=(dt_atual-td))['Close']

### Intervalo de datas fixas
# df = yf.download("BOVA11.SA", start="2020-03-03", end="2023-03-03")['Close']

[*********************100%***********************]  1 of 1 completed


In [10]:
df_teste_est

Date
2020-03-03    101.750000
2020-03-04    103.099998
2020-03-05     98.790001
2020-03-06     94.400002
2020-03-09     82.699997
                 ...    
2023-03-20     97.699997
2023-03-21     97.930000
2023-03-22     97.029999
2023-03-23     94.580002
2023-03-24     95.639999
Name: Close, Length: 762, dtype: float64

# **Teste de Estacionaridade KPSS (Kwiatkowski-Phillips-Schmidt-Shin)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [11]:
kpss_01 = statsmodels.tsa.stattools.kpss(df_teste_est)
print('Estatítica do teste: {:.4f}'.format(kpss_01[0]))
print('p_valor: {:.4f}'.format(kpss_01[1]))
print('número de lags: {}'.format(kpss_01[2]))
print('Valores Críticos:')
for chave, valor in kpss_01[3].items():
   print('{}: {:.4f}'.format(chave, valor))

Estatítica do teste: 1.0576
p_valor: 0.0100
número de lags: 17
Valores Críticos:
10%: 0.3470
5%: 0.4630
2.5%: 0.5740
1%: 0.7390


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


# **Teste de Estacionaridade DFuller (Dickey Fuller)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [12]:
dfuller_01 = statsmodels.tsa.stattools.adfuller(df_teste_est)
print('Estatítica do teste: {:.4f}'.format(dfuller_01[0]))
print('p_valor: {:.8f}'.format(dfuller_01[1]))
print('Valores Críticos:')
for chave, valor in dfuller_01[4].items():
   print('{}: {:.4f}'.format(chave, valor))

Estatítica do teste: -1.7492
p_valor: 0.40597583
Valores Críticos:
1%: -3.4390
5%: -2.8654
10%: -2.5688


# **Excluindo Outliers para o Teste de Estacionaridade**

In [13]:
df = df_teste_est.reset_index()

In [14]:
df

,Date,Close
0,2020-03-03,101.750000
1,2020-03-04,103.099998
2,2020-03-05,98.790001
3,2020-03-06,94.400002
4,2020-03-09,82.699997
...,...,...
757,2023-03-20,97.699997
758,2023-03-21,97.930000
759,2023-03-22,97.029999
760,2023-03-23,94.580002


In [15]:
df_bova_temp = df[['Date', 'Close']].rename(columns = {'Date': 'ds', 'Close': 'y'})

In [16]:
df_bova_temp

,ds,y
0,2020-03-03,101.750000
1,2020-03-04,103.099998
2,2020-03-05,98.790001
3,2020-03-06,94.400002
4,2020-03-09,82.699997
...,...,...
757,2023-03-20,97.699997
758,2023-03-21,97.930000
759,2023-03-22,97.029999
760,2023-03-23,94.580002


In [17]:
df_bova = pd.DataFrame({
  "ds": df_bova_temp['ds'].dt.date,
  "y":df_bova_temp['y']
})

In [19]:
dados = df_bova['y']
dados.describe()

count    762.000000
mean     104.013861
std       11.465052
min       61.139999
25%       98.352499
50%      105.830002
75%      111.497501
max      125.750000
Name: y, dtype: float64

In [20]:
boxplot_bova = px.box(dados, y="y")
boxplot_bova.show()

In [21]:
q1 = np.quantile(dados, .25, method="midpoint")
q2 = np.quantile(dados, .5, method="midpoint")
q3 = np.quantile(dados, .75, method="midpoint")
IIQ = q3 - q1

In [22]:
### Limite inferior do boxplot
limite_inferior = q1 - 1.5 * IIQ
limite_inferior

78.65999794006348

In [23]:
### Limite superior do boxplot
limite_superior = q3 + 1.5 * IIQ
limite_superior

131.1800022125244

In [25]:
df_bova_lim_inf = df_bova[df_bova['y']>limite_inferior]
df_bova_clean = df_bova_lim_inf[df_bova_lim_inf['y']<limite_superior]
df_bova = df_bova_clean
df_bova

,ds,y
0,2020-03-03,101.750000
1,2020-03-04,103.099998
2,2020-03-05,98.790001
3,2020-03-06,94.400002
4,2020-03-09,82.699997
...,...,...
757,2023-03-20,97.699997
758,2023-03-21,97.930000
759,2023-03-22,97.029999
760,2023-03-23,94.580002


In [28]:
df_teste_estacionaridade = pd.Series(df_bova['y'].values, index = df_bova['ds'])

# **Teste de Estacionaridade KPSS (Kwiatkowski-Phillips-Schmidt-Shin)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [29]:
kpss_02 = statsmodels.tsa.stattools.kpss(df_teste_estacionaridade)
print('Estatítica do teste: {:.4f}'.format(kpss_02[0]))
print('p_valor: {:.4f}'.format(kpss_02[1]))
print('número de lags: {}'.format(kpss_02[2]))
print('Valores Críticos:')
for chave, valor in kpss_02[3].items():
   print('{}: {:.4f}'.format(chave, valor))

Estatítica do teste: 0.5252
p_valor: 0.0360
número de lags: 17
Valores Críticos:
10%: 0.3470
5%: 0.4630
2.5%: 0.5740
1%: 0.7390


# **Teste de Estacionaridade DFuller (Dickey Fuller)**

**Ho = não é estacionário: estatística do teste > valor crítico**

**Ha = é estacionário: estatística do teste < valor crítico**

In [30]:
dfuller_02 = statsmodels.tsa.stattools.adfuller(df_teste_estacionaridade)
print('Estatítica do teste: {:.4f}'.format(dfuller_02[0]))
print('p_valor: {:.8f}'.format(dfuller_02[1]))
print('Valores Críticos:')
for chave, valor in dfuller_02[4].items():
   print('{}: {:.4f}'.format(chave, valor))

Estatítica do teste: -2.0739
p_valor: 0.25512862
Valores Críticos:
1%: -3.4395
5%: -2.8656
10%: -2.5689
